In [135]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np

In [136]:
df = pd.read_csv(r"E:\New folder (2)\DATAsets\titanic_dataset.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
353,354,0,3,"Arnold-Franchi, Mr. Josef",male,25.0,1,0,349237,17.8000,NaN,S
753,754,0,3,"Jonkoff, Mr. Lalio",male,23.0,0,0,349204,7.8958,NaN,S
807,808,0,3,"Pettersson, Miss. Ellen Natalia",female,18.0,0,0,347087,7.7750,NaN,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S
585,586,1,1,"Taussig, Miss. Ruth",female,18.0,0,2,110413,79.6500,E68,S


In [137]:
df = df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

In [138]:
from sklearn.model_selection import train_test_split
x_test,x_train,y_test,y_train = train_test_split(df.drop(columns=["Survived"]),df["Survived"],random_state=42,train_size=0.20)
y_train

709    1
439    0
840    0
720    1
39     1
      ..
639    0
878    0
824    0
803    1
619    0
Name: Survived, Length: 713, dtype: int64

In [139]:
#filling the missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [140]:
df["Sex"] = df["Sex"].astype("category") #memory optimisation

In [141]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy="most_frequent")

In [142]:
si_X_train_age_imputed = si_age.fit_transform(x_train[["Age"]])
si_X_train_embarked_ = si_embarked.fit_transform(x_train[["Embarked"]]) #i dont know the reason why double [[]] is used

In [143]:
x_test_imputed = si_age.transform(x_test[["Age"]])
x_test_imputed_embarked = si_embarked.transform(x_test[["Embarked"]])

0    0
dtype: int64

In [145]:
#concateneting the changes
x_train_age=pd.DataFrame(si_X_train_age_imputed)
x_train_embarked = pd.DataFrame(si_X_train_embarked_)
x_train.drop(columns=["Age"],inplace=True)
x_train["Age"] = si_X_train_age_imputed



In [146]:
x_test["Age"]= x_test_imputed
x_test["Embarked"]= pd.DataFrame(x_test_imputed_embarked)[0]

In [147]:
x_train["Embarked"] = x_train_embarked[0]

In [148]:
#applying one hot encoding
ohe_age = OneHotEncoder(handle_unknown="ignore",sparse_output=False)
ohe_embarked = OneHotEncoder(handle_unknown="ignore",sparse_output=False)


In [149]:
male_female_train = ohe_age.fit_transform(x_train[["Sex"]])
male_female_test = ohe_age.fit_transform(x_test[["Sex"]])


In [150]:
Embarked_train = ohe_age.fit_transform(x_train[["Embarked"]])
Embarked_test = ohe_age.fit_transform(x_test[["Embarked"]])

In [151]:
#in decision tree we dont have to remove one columns as there is no case of multiple collinearity
x_train.drop(columns=["Sex","Embarked"],inplace=True)
x_test.drop(columns=["Sex","Embarked"],inplace=True)

In [152]:
x_train_transformed = np.concatenate((x_train,male_female_train,Embarked_train),axis=1)
x_test_transformed = np.concatenate((x_test,male_female_test,Embarked_test),axis=1)

In [153]:
pd.DataFrame(x_train_transformed)

,0,1,2,3,4,5,6,7,8,9,10
0,3.0,1.0,1.0,15.2458,29.531933,0.0,1.0,0.0,0.0,1.0,0.0
1,2.0,0.0,0.0,10.5000,31.000000,0.0,1.0,0.0,0.0,1.0,0.0
2,3.0,0.0,0.0,7.9250,20.000000,0.0,1.0,0.0,0.0,0.0,1.0
3,2.0,0.0,1.0,33.0000,6.000000,1.0,0.0,0.0,0.0,0.0,1.0
4,3.0,1.0,0.0,11.2417,14.000000,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
708,3.0,1.0,0.0,16.1000,29.531933,0.0,1.0,0.0,0.0,1.0,0.0
709,3.0,0.0,0.0,7.8958,29.531933,0.0,1.0,0.0,0.0,0.0,1.0
710,3.0,4.0,1.0,39.6875,2.000000,0.0,1.0,0.0,0.0,0.0,1.0
711,3.0,0.0,1.0,8.5167,0.420000,0.0,1.0,0.0,0.0,0.0,1.0


In [154]:
#Model_bulding
dt = DecisionTreeClassifier()

In [155]:
dt.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [156]:
y_predict=dt.predict(x_test_transformed)

In [157]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.6910112359550562